### Import modules

In [1]:
from bs4 import BeautifulSoup
from requests import get
import re
import pandas as pd
import numpy as np
import json
import datetime

### Web scrape IMDB for list of movies being screened locally

In [2]:
moviesScreeningUrl = 'https://www.imdb.com/showtimes/location?'
moviesScreeningResponse = get(moviesScreeningUrl)
moviesScreening = BeautifulSoup(moviesScreeningResponse.text, 'html.parser')

# Selecting all movies on webpage
moviesScreening = moviesScreening.find_all('div',
                                           attrs={'class':'lister-item mode-grid'})

### Create list with data (movie title, user rating, Metascore, genres, release date) scraped from IMDB

In [3]:
moviesScreeningList = []
# Using regex to extract the movie release date from the webpage
datePattern = re.compile('[0-9]\d{1}.[A-z]\w{2}.[0-9)]\d{3}')
for movie in moviesScreening:
    title = movie.find_all('a')[1].string
    userRating = movie.find_all('span',
                                attrs={'name':'user_rating'})[0]
    userRating = str(userRating).strip('<span data-value="').strip('" name="user_rating"></span>')
    try:
        imdbGenres = movie.find_all('span',
                                attrs={'class':'genre'})[0].text.strip('\n')
    except:
        imdbGenres = None
#     Using these try blocks to prevent an error from breaking the program.
    releaseDate = movie.find_all('div',
                                 attrs={'id':'release_date'})[0].text
    try:
        releaseDate = datePattern.search(releaseDate).group(0)
    except:
        releaseDate = np.nan
    try:
        metaScore = movie.find_all('div',attrs={'class':'inline-block ratings-metascore'})[0]\
                                        .text.strip('        \n        Metascore\n            ')
    except:
        metaScore = np.nan
    moviesScreeningList.append([title,userRating,metaScore,imdbGenres,releaseDate])

### Web scrape Flixter for Rotten Tomatoes ratings

In [4]:
rtRatingsUrl = 'https://www.flixster.com/top-box-office'
rtRatingsResponse = get(rtRatingsUrl)
rtRatings = BeautifulSoup(rtRatingsResponse.text, 
                          'html.parser')

In [5]:
#Using regex to find section of code containing the movie data
rgPattern = re.compile(r"__NEXT_DATA__\s=\s(\{.*\});")
rtScript = rtRatings.find_all("script")
movieData = rgPattern.search(str(rtScript)).group(1)

# This section of the script is in JSON format,
# so I'm converting the text to JSON to make it easier to extract the data
movieData = json.loads(movieData)
movieData = movieData['props']['initialReduxState']['boxOffice']['api']['topBoxOfficeList']['data']

### Create list with data (movie title, critic score, audience score) scraped from Flixter

In [6]:
rtMovieList=[]
for movie in movieData:
    title = movie['title']
    try:
        criticScore = movie['tomatometer']['tomatometer']
#     See comment above about use of try blocks
    except:
        criticScore = np.nan
    try:
        audienceScore = movie['userRatingSummary']['dtlLikedScore']
    except:
        audienceScore = np.nan
    rtMovieList.append([title,criticScore,audienceScore])

### Create dataframes for analysis

In [7]:
# Creating a data frame with the IMDB data
screeningsDF = pd.DataFrame(data = moviesScreeningList,
                            columns=['Title','IMDB user rating','Metascore','Genres','Release date'])\
                            .set_index('Title')

# Creating a data frame with the Flixter data
ratingsDF = pd.DataFrame(data = rtMovieList,
                         columns=['Title','Critics score','Audience score'])\
                        .set_index('Title')

# Merging the above dataframes
moviesDF = screeningsDF.merge(ratingsDF, 
                              on='Title',
                              how='left')

### Update columns to the correct data type to allow for calculations

In [8]:
moviesDF['Critics score'] = moviesDF['Critics score'].astype(float)
moviesDF['Audience score'] = moviesDF['Audience score'].astype(float)
moviesDF['IMDB user rating'] = moviesDF['IMDB user rating'].astype(float)
moviesDF['Metascore'] = moviesDF['Metascore'].astype(float)
moviesDF['Release date'] = pd.to_datetime(moviesDF['Release date']).dt.date

### Filter the dataframe to select the top movies

In [27]:
moviesDF['Critically acclaimed'] = (((moviesDF['Critics score'] >= 85) 
                                     & (moviesDF.Metascore >= 80)) 
                                    | ((moviesDF['Critics score'].isna()) 
                                       & (moviesDF.Metascore >= 80)))
moviesDF['Critically acclaimed'] = np.where(moviesDF['Critically acclaimed'] == True,1,0)

moviesDF['Audience favorite'] = (((moviesDF['Audience score'] >= 85) 
                                  & (moviesDF['IMDB user rating'] >= 8)) 
                                    | ((moviesDF['Audience score'].isna()) 
                                       & (moviesDF['IMDB user rating'] >= 8)))
moviesDF['Audience favorite'] = np.where(moviesDF['Audience favorite'] == True,1,0)

topMovies = moviesDF[(moviesDF['Critically acclaimed'] == True) | (moviesDF['Audience favorite'] == True)].copy()

### Create boolean columns for movie genres and scoring

In [28]:
topMovies['Comedy'] = (topMovies['Genres'].str.contains('Comedy'))
topMovies['Comedy'] = np.where(topMovies['Comedy'] == True,1,0)

topMovies['Music'] = (topMovies['Genres'].str.contains('Music'))
topMovies['Music'] = np.where(topMovies['Music'] == True,1,0)

topMovies['Animation'] = (topMovies['Genres'].str.contains('Animation'))
topMovies['Animation'] = np.where(topMovies['Animation'] == True,1,0)

# Assigning a total score based on above boolean columns
topMovies['Total score'] = (topMovies['Critically acclaimed'] 
                            + topMovies['Audience favorite'] 
                            + topMovies['Comedy'] 
                            + topMovies['Music']
                            + topMovies['Animation'])

### List of top movies

In [29]:
# Sorting the data frame and highlighting the boolean columns that are True (value = 1)
topMovies[['Audience score',
           'Critics score',
           'IMDB user rating',
           'Metascore',
           'Critically acclaimed',
           'Audience favorite',
           'Comedy','Music',
           'Animation',
           'Total score']]\
            .sort_values(['Total score',
                          'Audience favorite',
                          'Critically acclaimed',
                          'Comedy',
                          'Music'],
                         ascending=False)\
                        .style.highlight_max(subset=['Critically acclaimed',
                                                      'Audience favorite',
                                                      'Comedy',
                                                      'Music',
                                                      'Animation'], 
                                             color='lightblue')

,Audience score,Critics score,IMDB user rating,Metascore,Critically acclaimed,Audience favorite,Comedy,Music,Animation,Total score
Title,,,,,,,,,,
Toy Story 4,94,97,8.1,84,1,1,1,0,1,4
Once Upon a Time in Hollywood,nan,nan,8,83,1,1,1,0,0,3
Parasite,nan,nan,8.5,92,1,1,1,0,0,3
The Farewell,87,99,8.1,90,1,1,1,0,0,3
Chhichhore,92,55,8.6,nan,0,1,1,0,0,2
Gang Leader,nan,nan,8.1,nan,0,1,1,0,0,2
Booksmart,nan,nan,7.3,84,1,0,1,0,0,2
Amazing Grace,nan,nan,7.6,94,1,0,0,1,0,2
Section 375,nan,nan,8.7,nan,0,1,0,0,0,1


### Movies loved by critics and audiences

In [24]:
topMovies[['Genres','Release date']]\
        [(topMovies['Critically acclaimed'] == 1) 
         & (topMovies['Audience favorite'] == 1)]\
        .sort_values(['Release date'],
                     ascending=False)

,Genres,Release date
Title,,
The Farewell,"Comedy, Drama",2019-09-05
Once Upon a Time in Hollywood,"Comedy, Drama",2019-08-15
Toy Story 4,"Animation, Adventure, Comedy",2019-06-20
Parasite,"Comedy, Drama, Thriller",2019-05-30


### Movies loved by critics but not audiences

In [25]:
topMovies[['Genres','Release date']]\
        [(topMovies['Critically acclaimed'] == 1) 
         & (topMovies['Audience favorite'] == 0)]\
        .sort_values(['Release date'],
                     ascending=False)

,Genres,Release date
Title,,
Amazing Grace,"Documentary, Music",2019-08-29
Booksmart,Comedy,2019-06-27


### Movies loved by audiences but not critics

In [26]:
topMovies[['Genres','Release date']]\
        [(topMovies['Critically acclaimed'] == 0) 
         & (topMovies['Audience favorite'] == 1)]\
        .sort_values(['Release date'],
                     ascending=False)

,Genres,Release date
Title,,
Section 375,"Crime, Drama",2019-09-13
Gang Leader,Comedy,2019-09-13
Chhichhore,"Comedy, Drama",2019-09-06
Danger Close: The Battle of Long Tan,"Action, Drama, War",2019-08-08
